# Imports

In [28]:
import numpy as np
import _pickle as pickle
import gym
import time
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.initializers as initializers
 # CHANGED
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Model

In [29]:
#hello
env = gym.make("Pong-v0")
initializer = initializers.GlorotNormal

x_in = tf.keras.layers.Input(shape = (6400,))
x = tf.keras.layers.Dense(200, kernel_initializer= initializer, activation="relu")(x_in)
x_out = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(x_in, x_out)

model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["acc"])

In [30]:
# weights = x.get_weights()
# print(weights)

In [31]:
env.observation_space

Box(0, 255, (210, 160, 3), uint8)

In [32]:
env.unwrapped.get_action_meanings()

# NOOP is the same as FIRE (standing still)
# LEFT is the same as LEFTFIRE (down)
# RIGHT is the same as RIGHTFIRE (up)

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [33]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 6400)]            0         
_________________________________________________________________
dense_4 (Dense)              (None, 200)               1280200   
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 201       
Total params: 1,280,401
Trainable params: 1,280,401
Non-trainable params: 0
_________________________________________________________________


# Preprocessing

In [34]:
def prepro(input_frame):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
    input_frame = input_frame[34:194] # crop
    input_frame = input_frame[::2,::2,0] # downsample by factor of 2 (halves the resolution of the image)
    #This takes every other pixel in the image
    input_frame[input_frame == 144] = 0 # erase background (background type 1)
    input_frame[input_frame == 109] = 0 # erase background (background type 2)
    input_frame[input_frame != 0] = 1 # everything else (paddles, ball) just set to 1
    return input_frame.astype(np.float).ravel()

# Discounted Rewards

In [35]:
def discount_rewards(rewards):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(rewards)
    running_add = 0
    for t in reversed(range(0, rewards.size)):
        if rewards[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + rewards[t]
        discounted_r[t] = running_add
    return discounted_r

# Hyperparamters

In [36]:
render = False
prev_frame = None
game_dimensions = 80*80
gamma = 0.99
resume = False
batch_size = 10

# Main Code

In [38]:
observation = env.reset()
reward_sum = 0
eps = 1
episode_number = 0
running_reward = None
ep_observations, ep_rewards, ep_gradient_log_ps, ep_hidden_layer_values, ep_up_probs = [], [], [], [], []

if resume:
    model.load_weights("ModelWeights")

while True:
    if render:
        env.render()
    
    curr_frame = prepro(observation)
    change_in_frame = curr_frame - prev_frame if prev_frame is not None else np.zeros(game_dimensions)
    prev_frame = curr_frame
    ep_observations.append(change_in_frame)

    # CHANGED 
    # policyforward is happening here
    
   # hidden_layer_values = np.dot(model.get_weights()[0].T, change_in_frame)
    # hidden_layer_values = tf.keras.activations.relu(hidden_layer_values)
        
    with tf.GradientTape(watch_accessed_variables=False) as tape:
        change_in_frame = change_in_frame.reshape((1,6400))
        change_in_frame = tf.convert_to_tensor(change_in_frame)
        tape.watch(change_in_frame)
        
    up_prob = model.predict(change_in_frame)
    ep_up_probs.append(up_prob)
    # up_prob = tf.convert_to_tensor(up_prob)

    action = 2 if np.random.uniform() < up_prob else 3 # roll the dice!

    # if np.random.random() < eps:
        # action = np.random.randint(2, 4)
    # else:
       # change_in_frame = change_in_frame.reshape((1,6400))
       # up_prob = model.predict(change_in_frame)
        # print(up_prob)
        #if up_prob >= .5:
           # action = 2
       # else:
           # action = 3
    
    observation, reward, done, _ = env.step(action) 
    
    y = 1 if action == 2 else 0
    
    ep_gradient_log_ps.append(y - up_prob)
        
   # ep_hidden_layer_values.append(hidden_layer_values)
    ep_rewards.append(reward)
    reward_sum += reward
    eps -= 0.001
    
    if done:
    
       # comb_ep_observations = np.vstack(ep_observations)
        #comb_ep_gradient_log_ps = []
        #for i in ep_gradient_log_ps:
           # comb_ep_gradient_log_ps.append([i])
            
        comb_ep_gradient_log_ps = np.vstack(ep_gradient_log_ps)
        
        #comb_ep_up_probs = []
        #for i in ep_up_probs:
            #comb_ep_up_probs.append(i)
        
        comb_ep_up_probs = np.vstack(ep_up_probs)
        comb_ep_rewards = np.vstack(ep_rewards)
        
       # comb_ep_hidden_layer_values = np.vstack(ep_hidden_layer_values)
        
        ep_observations, ep_gradient_log_ps, ep_rewards, ep_hidden_layer_values, ep_up_probs  = [], [], [], [], []
        
        discounted_comb_ep_rewards = discount_rewards(comb_ep_rewards)
        discounted_comb_ep_rewards -= np.mean(discounted_comb_ep_rewards) 
        discounted_comb_ep_rewards /= np.std(discounted_comb_ep_rewards)
        
        comb_ep_gradient_log_ps *= discounted_comb_ep_rewards
        comb_ep_gradient_log_ps = tf.convert_to_tensor(comb_ep_gradient_log_ps)
        
        #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=comb_ep_gradient_log_ps, logits=comb_ep_up_probs))
        #updates = tf.train.AdamOptimizer(0.001).minimize(cost)
        #sess.run(updates)
        
        # model.fit(comb_ep_observations, comb_ep_up_probs, 10, 50)
        # grad = tape.gradient(comp_ep_gradient_log_ps, comp_ep_hidden_layer_values)
        
        running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
        print ('resetting env. episode reward total was %f. running mean: %f' % (reward_sum, running_reward))
        reward_sum = 0
        observation = env.reset() # reset env
        prev_frame = None
        episode_number += 1
        
        #if episode_number % batch_size == 0:
            
            # Back Propagation Code Goes Here
           # model.save_weights("ModelWeights")
            
    if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
        print('ep %d: game finished, reward: %f' % (episode_number, reward) + ('' if reward == -1 else ' !!!!!!!!'))
        

ValueError: When using data tensors as input to a model, you should specify the `steps` argument.

In [14]:
# np.unique(comb_ep_gradient_log_ps)
print(len(comb_ep_gradient_log_ps))
len(np.unique(comb_ep_gradient_log_ps))

1484


1483